# Calculating the Discrete Fréchet Distance
The discrete Fréchet distance measures the similarity between two polygonal curves or poly-lines.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

from numba import jit
from distances.discrete import DiscreteFrechet, LinearDiscreteFrechet, VectorizedDiscreteFrechet, FastDiscreteFrechet
from distances.discrete import euclidean

%matplotlib inline

In [2]:
np.set_printoptions(precision=4)

Declare the test poly-lines.

In [3]:
p = np.array([[0.2, 2.0], 
              [1.5, 2.8], 
              [2.3, 1.6], 
              [2.9, 1.8], 
              [4.1, 3.1], 
              [5.6, 2.9], 
              [7.2, 1.3],
              [8.2, 1.1]])

In [4]:
q = np.array([[0.3, 1.6], 
              [3.2, 3.0], 
              [3.8, 1.8],  
              [5.2, 3.1], 
              [6.5, 2.8], 
              [7.0, 0.8],
              [8.9, 0.6]])

In [5]:
frechet3 = LinearDiscreteFrechet(euclidean)

In [9]:
%%timeit
frechet3.distance(p, q)

19.3 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
frechet0 = DiscreteFrechet(euclidean)

In [10]:
%%timeit
frechet0.distance(p, q)

169 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
frechet1 = VectorizedDiscreteFrechet(euclidean)

In [15]:
%%timeit
frechet1.distance(p, q)

140 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
frechet2 = FastDiscreteFrechet(euclidean)

In [16]:
%%timeit
frechet2.distance(p, q)

26.2 µs ± 909 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
frechet0.distance(p, q)

1.6970562748477138

In [18]:
frechet1.distance(p, q)

1.6970562748477138

In [19]:
frechet2.distance(p, q)

1.6970562748477138

In [20]:
frechet3.distance(p, q)

1.6970562748477138

In [ ]:
@jit(nopython=True)
def bresenham_pairs(x0: int, y0: int, x1: int, y1: int) -> np.ndarray:
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    dim = max(dx, dy)
    i = 0
    p = np.zeros((dim, 2), dtype=np.int32)
    x, y = x0, y0
    sx = -1 if x0 > x1 else 1
    sy = -1 if y0 > y1 else 1
    if dx > dy:
        err = dx // 2
        while x != x1:
            p[i] = np.array([x, y])
            i += 1
            err -= dy
            if err < 0:
                y += sy
                err += dx
            x += sx
    else:
        err = dy // 2
        while y != y1:
            p[i] = np.array([x, y])
            i += 1
            err -= dx
            if err < 0:
                x += sx
                err += dy
            y += sy        
    return p

In [ ]:
%%timeit
bp = bresenham_pairs(0, 0, n_p, n_q)

In [ ]:
diag = [(e[0], e[1]) for e in bp]

In [ ]:
p[bp[:,0]]

In [ ]:
q[bp[:,1]]

In [ ]:
bp

In [ ]:
def euclidean_pairwise(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    d = p - q
    return np.sqrt(d[:,0] ** 2 + d[:,1] ** 2)

In [ ]:
d = p[0] - q[0]

In [ ]:
d

In [ ]:
np.array([p[0]])

In [ ]:
euclidean_pairwise(np.array([p[0]]), np.array([q[0]]))[0]

In [ ]:
epq = euclidean_pairwise(p[bp[:,0]], q[bp[:,1]])

In [ ]:
epq

In [ ]:
diag_max = epq.max()

In [ ]:
n_p = p.shape[0]
n_q = q.shape[0]

In [ ]:
dist = np.zeros((n_p, n_q))
dist.fill(-1.0)
#dist.fill(np.infty)

In [ ]:
dist

In [ ]:
for i in range(bp.shape[0]):
    dist[bp[i][0], bp[i][1]] = epq[i]

In [ ]:
dist

In [ ]:
#%%timeit
for k in range(bp.shape[0] - 1):
    ij = bp[k]
    i0 = ij[0]
    j0 = ij[1]
    
    for i in range(i0 + 1, n_p):
        if dist[i, j0] == -1:
            d = point_euclidean(p[i], q[j0])
            if d < diag_max:
                dist[i, j0] = d
            else:
                break
    
    for j in range(j0 + 1, n_q):
        if dist[i0, j] == -1:
            d = point_euclidean(p[i0], q[j])
            if d < diag_max:
                dist[i0, j] = d
            else:
                break            

In [ ]:
def calculate_distance_matrix(p, q, dist_func):
    n_p = p.shape[0]
    n_q = q.shape[0]
    
    bp = bresenham_pairs(0, 0, n_p, n_q)
    diag = [(e[0], e[1]) for e in bp]
    epq = euclidean_pairwise(p[bp[:,0]], q[bp[:,1]])
    
    dist = np.zeros((n_p, n_q))
    dist.fill(-1.0)

In [ ]:
n_p, n_q

In [ ]:
dist

In [ ]:
def get_min(dist, i, j):
    if i == 0 and j == 0:
        a = np.array([dist[i, j]])
    elif i == 0:
        a = np.array([dist[i, j-1]])
    elif j == 0:
        a = np.array([dist[i-1, j]])
    else:
        a = np.array([dist[i-1, j-1], dist[i, j-1], dist[i-1, j]])
        np.place(a, a == -1, np.inf)
    return a.min()

In [ ]:
get_min(dist, 2, 1)

In [ ]:
%%timeit
f = dist.copy()

for k in range(bp.shape[0]):
    ij = bp[k]
    i0 = ij[0]
    j0 = ij[1]
    
    for i in range(i0, n_p):
        d = dist[i, j0]
        if d != -1:
            f[i, j0] = max(d, get_min(f, i, j0))
        else:
            break
    
    for j in range(j0, n_q):
        d = dist[i0, j]
        if d != -1:
            f[i0, j] = max(d, get_min(f, i0, j))
        else:
            break

In [ ]:
f

In [ ]:
f[n_p-1, n_q-1]

In [ ]:
def calculate_frechet_matrix(d):
    f = np.zeros_like(d)
    
    return f

The `euclidean` function calculates the distance between two points in *R*<sup>N</sup>

In [ ]:
def euclidean_pairwise(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    d = p - q
    return np.sqrt(d[:,0] ** 2 + d[:,1] ** 2)

In [ ]:
def np_euclidean(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    """
    Calculates the point-to-point distance between poly-lines p and q
    :param p: Poly-line p
    :param q: Poly-line q
    :return: Distance array
    """
    n_p = p.shape[0]
    n_q = q.shape[0]
    pp = np.repeat(p, n_q, axis=0)
    qq = np.tile(q, (n_p, 1))
    dd = pp - qq
    dist = np.sqrt(dd[:, 0] ** 2 + dd[:, 1] ** 2).reshape(n_p, n_q)
    return dist

In [ ]:
frechet = Frechet(np_euclidean)

In [ ]:
#%%timeit
frechet.distance(p, q)

In [ ]:
fsd = frechet.ca

In [ ]:
pp = plt.imshow(fsd)

In [ ]:
fsd

In [ ]:
b = (fsd <= 3.6)

In [ ]:
pp = plt.imshow(b)

In [ ]:
dist = np.zeros((p.shape[0], q.shape[0]))

In [ ]:
fsd

In [ ]:
#%%timeit
n_p = p.shape[0]
n_q = q.shape[0]
pp = np.repeat(p, n_q, axis=0)
qq = np.tile(q, (n_p, 1))
dd = pp - qq
dist = np.sqrt(dd[:, 0] ** 2 + dd[:, 1] ** 2).reshape(n_p, n_q)
fd = np.max(np.concatenate((dist.min(axis=1), (dist.min(axis=0)))))

In [ ]:
bp = bresenham_pairs(0, 0, n_p, n_q)

In [ ]:
bp

In [ ]:
p[bp[:,0]]

In [ ]:
q[bp[:,1]]

In [ ]:
ep = euclidean_pairwise(p[bp[:,0]], q[bp[:,1]])

In [ ]:
ep